In [89]:
from sympy import *
import sympy as sp
from opt_einsum import contract as _contract
import numpy as np
from dataclasses import dataclass

def contract(expr,*tensors,**args):
    if contract.dest is not None:
        expr+='->'+contract.dest
    rtval= _contract(expr,*tensors, backend='object',**args)
    return rtval.item() if rtval.shape==() else Array(rtval)
class contract_dest:
    def __init__(self, dest):
        self.dest= dest
    def __enter__(self):
        contract.dest,self.dest= self.dest,contract.dest
    def __exit__(self, *args):
        contract.dest= self.dest
contract.dest=None

In [289]:


class CoordinateTransformation:
    def __init__(self,coords1:list,coords2:list,coords2_in_1:dict,coords1_in_2:dict=None):
        self.coords1=coords1
        self.coords2=coords2
        self.coords2_in_1=coords2_in_1
        if coords1_in_2 is None:
            subs1_in_2=solve([a-b for a,b in coords2_in_1.items()],coords1,dict=True)
            coords1_in_2={x:x for x in coords1}
            print('try to solve for coords1_in_2')
            print(subs1_in_2)
            if len(subs1_in_2)>1:
                print('Warning: more than one solution found, use the first one')
            coords1_in_2.update(subs1_in_2[0])
        self.coords1_in_2=coords1_in_2
        self.jacobi=Matrix(together(derive_by_array(Array(coords2).subs(coords2_in_1),coords1))).T
        self.jacobi_inv=Matrix(together(derive_by_array(Array(coords1).subs(coords1_in_2),coords2))).T
    def transform(self,tensor,indice_positions='',inverse=False):
        if not inverse:
            jacobi,jacobi_inv,subs=self.jacobi,self.jacobi_inv,self.coords1_in_2
        else:
            jacobi,jacobi_inv,subs=self.jacobi_inv,self.jacobi,self.coords2_in_1
        for i,ind in enumerate(indice_positions):
            if ind=='u':
                # V^M=J^M_u v^u
                tensor=np.swapaxes(contract('Mu,u...->M...',jacobi,np.swapaxes(tensor,0,i)),0,i)
            elif ind=='d':
                # V_M=v_u (J^-1)^u_M
                tensor=np.swapaxes(contract('uM,u...->M...',jacobi_inv,np.swapaxes(tensor,0,i)),0,i)
        if isinstance(tensor,np.ndarray):
            tensor=Array(tensor)
        tensor=tensor.subs(subs)
        tensor=together(tensor)
        return tensor
    def inverse_transform(self,*args,**kwargs):
        return self.transform(*args,inverse=True,**kwargs)

In [290]:
r,theta,phi=coords1=sp.symbols('r theta phi',real=True)
x,y,z=coords2=sp.symbols('x y z',real=True)
coords2_in_1={x:r*sp.sin(theta)*sp.cos(phi),y:r*sp.sin(theta)*sp.sin(phi),z:r*sp.cos(theta)}
coords1_in_2={r:sp.sqrt(x**2+y**2+z**2),theta:sp.acos(z/sp.sqrt(x**2+y**2+z**2)),phi:sp.atan2(y,x)}
transf=CoordinateTransformation(coords1,coords2,coords2_in_1,coords1_in_2)

display(simplify((transf.jacobi@transf.jacobi_inv).subs(transf.coords1_in_2)))
display(simplify(transf.inverse_transform(transf.transform(r))))

Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 1]])

Abs(r)

In [302]:
def get_lie_derivative(coords):
    def lie_derivative(f,X,indice_positions=''):
        dX=derive_by_array(X,coords)
        # L_X f = X^u \partial_u f
        rtval=contract('i,i...->...',X,derive_by_array(f,coords))
        for i,ind in enumerate(indice_positions):
            if ind=='u':
                # L_X f^u -= (∂_v X^u) f^v
                rtval-=np.swapaxes(contract('vu,v...->u...',dX,np.swapaxes(f,0,i)),0,i)
            elif ind=='d':
                # L_X f_u += (∂_u X^v) f_v
                rtval+=np.swapaxes(contract('uv,v...->u...',dX,np.swapaxes(f,0,i)),0,i)
        if isinstance(rtval,np.ndarray):
            rtval=Array(rtval)
        return together(rtval)
    def lie_bracket(X,Y):
        return lie_derivative(Y,X,'u')
    return lie_derivative,lie_bracket

In [305]:
t,r,th,phi=coords=symbols('t,r,theta,phi')
f=Function('f')(r)
g=diag(-f,1/f,r**2,r**2*sin(th)**2)

display(g)
lie_derivative,lie_bracket=get_lie_derivative(coords)
display(lie_derivative(g,Array([1,0,0,0]),'dd'))
display(lie_derivative(g,Array([0,0,0,1]),'dd'))
display(trigsimp(lie_derivative(g,Array([0,0,sin(phi),cot(th)*cos(phi)]),'dd')))
display(trigsimp(lie_derivative(g,Array([0,0,cos(phi),-cot(th)*sin(phi)]),'dd')))

Matrix([
[-f(r),      0,    0,                  0],
[    0, 1/f(r),    0,                  0],
[    0,      0, r**2,                  0],
[    0,      0,    0, r**2*sin(theta)**2]])

[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]

[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]

[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]

[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]

In [306]:

coords=u,v,rho=sp.symbols('u,v,rho',real=True)

coords0=rho,t,phi=sp.symbols('rho,t,phi',real=True)
transf={u:(t+phi)/2,v:(t-phi)/2,rho:rho}
transf=CoordinateTransformation(coords0,coords,transf)

AdSRad=symbols('l')
g=AdSRad**2*diag(1,-exp(2*rho)/4,exp(2*rho)/4)
display(g)
g=transf.transform(g,'dd')
display(g)

try to solve for coords1_in_2
[{t: u + v, phi: u - v}]


Matrix([
[l**2,                  0,                 0],
[   0, -l**2*exp(2*rho)/4,                 0],
[   0,                  0, l**2*exp(2*rho)/4]])

[[0, -l**2*exp(2*rho)/2, 0], [-l**2*exp(2*rho)/2, 0, 0], [0, 0, l**2]]

In [307]:
m,n=symbols('m n',integer=True)

def get_zeta(n):
    return exp(I*n*u)*Array([1,-Rational(1,2)*n**2*exp(-2*rho),-I*n/2])


lie_derivative,lie_bracket=get_lie_derivative(coords)

display(factor(I*lie_bracket(get_zeta(n),get_zeta(m))))
display(factor((n-m)*get_zeta(n+m)))

[-(m - n)*exp(I*m*u)*exp(I*n*u), (m - n)*(m + n)**2*exp(-2*rho)*exp(I*m*u)*exp(I*n*u)/2, I*(m - n)*(m + n)*exp(I*m*u)*exp(I*n*u)/2]

[-(m - n)*exp(I*m*u)*exp(I*n*u), (m - n)*(m + n)**2*exp(-2*rho)*exp(I*m*u)*exp(I*n*u)/2, I*(m - n)*(m + n)*exp(I*m*u)*exp(I*n*u)/2]

In [308]:
dzndg=lie_derivative(g,get_zeta(n),'dd')
display(dzndg)

[[I*l**2*n**3*exp(I*n*u)/2, 0, 0], [0, 0, 0], [0, 0, 0]]